In [21]:
import os
import mne
import torch
import pickle
import torchaudio
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset
import import_ipynb
from utils import load_eeg_data
from utils import load_eye_data
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

## Load Test Data

In [22]:
eye_movement_features = './dataset/Eye_movement_features'
eeg_features = './dataset/EEG_DE_features'

In [23]:
eeg_dir = eeg_features
eye_dir = eye_movement_features
file_list = os.listdir(eeg_dir)
file_list.sort()

res_dir = './res/cv3/'
if not os.path.exists(res_dir):
    os.makedirs(res_dir)
cv = 1

for f_id in file_list:
    print(f"******************** {f_id} ********************")
    print("EEG DATA")
    load_eeg_data(eeg_dir, f_id, True)
    print("EYE TRACKING DATA")
    load_eye_data(eye_dir, f_id, True)

******************** 1_123.npz ********************
EEG DATA
[[11.08252158  8.91598993  7.89408803 ...  7.07195532  5.65708081
   4.36150606]
 [11.08181639  8.91510401  7.89364634 ...  7.07177633  5.6569868
   4.36162375]
 [11.08113882  8.91424457  7.89311581 ...  7.07146188  5.65674597
   4.36177956]
 ...
 [11.09093681  8.90651219  7.88062322 ...  7.05494398  5.64746577
   4.36830008]
 [11.09181965  8.90597236  7.87993625 ...  7.05439539  5.64685332
   4.36912144]
 [11.09237631  8.90556688  7.87957235 ...  7.05379577  5.64648296
   4.36975357]]
	Session 1 - Clip #1 ----> Happy
	Session 1 - Clip #2 ----> Fear
	Session 1 - Clip #3 ----> Neutral
	Session 1 - Clip #4 ----> Sad
	Session 1 - Clip #5 ----> Disgust
	Session 1 - Clip #6 ----> Happy
	Session 1 - Clip #7 ----> Fear
	Session 1 - Clip #8 ----> Neutral
	Session 1 - Clip #9 ----> Sad
	Session 1 - Clip #10 ----> Disgust
	Session 1 - Clip #11 ----> Happy
	Session 1 - Clip #12 ----> Fear
	Session 1 - Clip #13 ----> Neutral
	Session 1 -

## Load Train Data

In [24]:
eeg_raw = './dataset/EEG_raw'
eye_raw = './dataset/Eye_raw'

eeg_dir = eeg_raw
eye_dir = eye_raw
file_list = os.listdir(eeg_dir)
file_list.sort()

res_dir = './res/cv3/'
if not os.path.exists(res_dir):
    os.makedirs(res_dir)

In [25]:
# EEG RAW DATA
for f_id in file_list:
    print(f_id)
    eeg_raw = mne.io.read_raw_cnt(f'./dataset/EEG_raw/{f_id}')
    print(eeg_raw)
    print('\n')
    print(eeg_raw.info)

1_1_20180804.cnt
<RawCNT | 1_1_20180804.cnt, 66 x 3378360 (3378.4 s), ~81 kB, data not loaded>


<Info | 9 non-empty values
 bads: []
 ch_names: FP1, FPZ, FP2, AF3, AF4, F7, F5, F3, F1, FZ, F2, F4, F6, F8, ...
 chs: 66 EEG
 custom_ref_applied: False
 dig: 69 items (3 Cardinal, 66 EEG)
 highpass: 0.0 Hz
 lowpass: 500.0 Hz
 meas_date: 2018-04-08 17:35:05 UTC
 nchan: 66
 projs: []
 sfreq: 1000.0 Hz
 subject_info: 5 items (dict)
>


In [26]:
ch_names = eeg_raw.ch_names

# drop non-used channels
useless_ch = ['M1', 'M2', 'VEO', 'HEO']
eeg_raw.drop_channels(useless_ch)
new_ch = eeg_raw.ch_names
print(new_ch)
print(len(new_ch))

['FP1', 'FPZ', 'FP2', 'AF3', 'AF4', 'F7', 'F5', 'F3', 'F1', 'FZ', 'F2', 'F4', 'F6', 'F8', 'FT7', 'FC5', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'FC6', 'FT8', 'T7', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4', 'C6', 'T8', 'TP7', 'CP5', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'CP6', 'TP8', 'P7', 'P5', 'P3', 'P1', 'PZ', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO5', 'PO3', 'POZ', 'PO4', 'PO6', 'PO8', 'CB1', 'O1', 'OZ', 'O2', 'CB2']
62


## Visualize Data with MNE

In [27]:
x = eeg_raw.plot()
# plt.savefig("./images/EEG_raw_data.jpg")

In [28]:
data_matrix = eeg_raw.get_data()
print(data_matrix)

[[-4.93824482e-05 -4.80115414e-05 -4.97996807e-05 ...  1.78605318e-04
   1.67906284e-04  1.55389309e-04]
 [-4.86969948e-05 -5.03957272e-05 -5.25712967e-05 ...  1.54405832e-04
   1.47789717e-04  1.38103962e-04]
 [-6.88433647e-05 -7.08997250e-05 -7.05718994e-05 ...  1.60545111e-04
   1.58250332e-04  1.51783228e-04]
 ...
 [-2.04741955e-05 -2.02655792e-05 -2.08616257e-05 ... -1.18255615e-04
  -1.06900930e-04 -8.60989094e-05]
 [-2.67922878e-05 -2.62856483e-05 -2.57194042e-05 ... -5.71608543e-05
  -5.07831573e-05 -3.39746475e-05]
 [ 2.27987766e-05  2.33948231e-05  2.43484974e-05 ... -3.70323658e-04
  -3.55124474e-04 -3.30239534e-04]]


## Timestamp EEG

In [29]:
# Session #3
start_second = [30, 353, 478, 674, 825, 908, 1200, 1346, 1451, 1711, 2055, 2307, 2457, 2726, 2888]
end_second = [321, 418, 643, 764, 877, 1147, 1284, 1418, 1679, 1996, 2275, 2425, 2664, 2857, 3066]
sample_freq = 1000

data_trial_1 = data_matrix[:, start_second[0]*1000 : end_second[0]*1000]
data_trial_5 = data_matrix[:, start_second[4]*1000 : end_second[4]*1000]
data_trial_15 = data_matrix[:, start_second[14]*1000 : end_second[14]*1000]

# print(data_trial_1)
# print(data_trial_5)
# print(data_trial_15)

## Process Data

In [30]:
eeg_raw = './dataset/EEG_raw/1_1_20180804.cnt'
eye_raw = './dataset/Eye_raw/Session_1/10_1_20180507.xlsx'

### DataLoader

In [31]:
class myDataset(Dataset):
    def __init__(self, eeg_raw, eye_raw):
        self.eeg_data = mne.io.read_raw_cnt(eeg_raw)
        self.eye_data = pd.read_excel(eye_raw)

    def __len__(self):
        # return len(self.eeg_data)
        return min(len(self.eeg_data), len(self.eye_data))

    def __getitem__(self, index):
        eeg_channel_name = 'FP1'  
        eeg_signal = self.eeg_data[eeg_channel_name][0][0]

        eye_signal = self.eye_data.iloc[index, 0]

        eeg_signal_tensor = torch.tensor(eeg_signal)
        eye_signal_tensor = torch.tensor(eye_signal)

        eeg_mean = torch.mean(eeg_signal_tensor)
        eeg_std = torch.std(eeg_signal_tensor)
        eye_mean = torch.mean(eye_signal_tensor)
        eye_std = torch.std(eye_signal_tensor)

        # Normalize the signal data
        eeg_signal_tensor = (eeg_signal_tensor - eeg_mean) / eeg_std
        eye_signal_tensor = (eye_signal_tensor - eye_mean) / eye_std

        return eeg_signal_tensor, eeg_mean, eeg_std, eye_signal_tensor, eye_mean, eye_std

# Create a dataset from the EEG and eye files
dataset = myDataset(eeg_raw, eye_raw)
# Create a data loader for the dataset
data_loader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

##### Samples

In [32]:
# dataset.shape = 151 samples
# tamaño de cada sample:  6 (eeg_signal_tensor, eeg_mean, eeg_std, eye_signal_tensor, eye_mean, eye_std)

# print(dataset[0])

In [33]:
# iterate over data_loader

# next(iter(data_loader))

## Plot EEG_signal_tensor 

In [34]:
import matplotlib.pyplot as plt

channel = 'FP1'

# Assuming your tensor is named 'eeg_tensor' and has shape (3378360,)
eeg_channel_data = dataset[0][0]

# Create a time vector corresponding to the data
time = range(len(eeg_channel_data))
time = [t/1000 for t in time]  # Convert to seconds

# Create a line plot of the EEG channel data
plt.plot(time, eeg_channel_data)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (uV)')
plt.title('EEG data of channel FP1')
plt.show()

C:\Users\scago\AppData\Local\Temp\ipykernel_15960\1645269431.py:17: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


## LSTM

In [35]:
import torch
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).requires_grad_()
        # Initialize cell state with zeros
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).requires_grad_()
        # 1st time-step
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        # output layer
        out = self.fc(out[:, -1, :]) 
        return out


In [36]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

# Define hyperparameters
input_size = 28
hidden_size = 128
num_layers = 2
output_size = 10
learning_rate = 0.01
num_epochs = 10
batch_size = 100

# Load MNIST dataset
train_dataset = dsets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = dsets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Create LSTM model
lstm = LSTM(input_size, hidden_size, num_layers, output_size)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape input data
        images = images.view(-1, input_size, input_size)
        # Forward pass
        outputs = lstm(images)
        loss = criterion(outputs, labels)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Print loss and accuracy
        if (i+1) % 100 == 0:
            total = labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == labels).sum().item()
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.item(), (correct/total)*100))

# Evaluate the model
lstm.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.view(-1, input_size, input_size)
        outputs = lstm(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy: {:.2f}%'.format((correct/total)*100))


Epoch [1/10], Step [100/600], Loss: 0.4869, Accuracy: 85.00%
Epoch [1/10], Step [200/600], Loss: 0.1325, Accuracy: 96.00%
Epoch [1/10], Step [300/600], Loss: 0.2054, Accuracy: 92.00%
Epoch [1/10], Step [400/600], Loss: 0.2046, Accuracy: 95.00%
Epoch [1/10], Step [500/600], Loss: 0.1024, Accuracy: 96.00%
Epoch [1/10], Step [600/600], Loss: 0.1607, Accuracy: 93.00%
Epoch [2/10], Step [100/600], Loss: 0.0852, Accuracy: 98.00%
Epoch [2/10], Step [200/600], Loss: 0.1344, Accuracy: 95.00%
Epoch [2/10], Step [300/600], Loss: 0.0259, Accuracy: 100.00%
Epoch [2/10], Step [400/600], Loss: 0.1060, Accuracy: 95.00%
Epoch [2/10], Step [500/600], Loss: 0.0570, Accuracy: 99.00%
Epoch [2/10], Step [600/600], Loss: 0.0920, Accuracy: 97.00%
Epoch [3/10], Step [100/600], Loss: 0.0715, Accuracy: 98.00%
Epoch [3/10], Step [200/600], Loss: 0.0128, Accuracy: 100.00%
Epoch [3/10], Step [300/600], Loss: 0.0666, Accuracy: 96.00%
Epoch [3/10], Step [400/600], Loss: 0.0230, Accuracy: 100.00%
Epoch [3/10], Step [5

In [37]:
'''
    Pruebaaaaaa (sin usar SEED-V dataset)
'''

import torch
import torch.nn as nn
# import torchvision.datasets as dsets
import torchvision.transforms as transforms

# Define hyperparameters
input_size = 28
hidden_size = 128
num_layers = 2
output_size = 10
learning_rate = 0.01
num_epochs = 10
batch_size = 100

# Load MNIST dataset
# train_dataset = dsets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
# test_dataset = dsets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)
# train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
# test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
for f_id in file_list:
    train_dataset =  # dsets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
    test_dataset = load_eeg_data(eeg_dir, f_id, False)
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Create LSTM model
lstm = LSTM(input_size, hidden_size, num_layers, output_size)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape input data
        images = images.view(-1, input_size, input_size)
        # Forward pass
        outputs = lstm(images)
        loss = criterion(outputs, labels)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Print loss and accuracy
        if (i+1) % 100 == 0:
            total = labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == labels).sum().item()
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.item(), (correct/total)*100))

# Evaluate the model
lstm.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.view(-1, input_size, input_size)
        outputs = lstm(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy: {:.2f}%'.format((correct/total)*100))


SyntaxError: invalid syntax (873592335.py, line 25)

## WaveNet

## WaveNet

## Transformers